In [1]:
import numpy as np
import pandas as pd
#import cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os


from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
MAX_SENT_LENGTH = 60
MAX_SENTS = 15
MAX_NB_WORDS = 80000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.0


def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

In [9]:
# Prepare the vocabulary
data_train = pd.read_csv('/Users/venkatakrishnamohansunkara/Desktop/DM/cleaned_balanced_all_data.csv', sep=',')
print(data_train.shape)

data_train.dropna(inplace=True)

(88042, 5)


In [10]:
from nltk import tokenize

sentence_list = []
texts = []
i=0

for idx in data_train['Unnamed: 0']:
    i = idx
    text = BeautifulSoup(data_train.content[idx])
    text = clean_str(text.get_text())
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sentence_list.append(sentences)

    #labels.append(data_train.cat_id[idx])

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/anaconda3/lib/python3.6/site-packages/keras/preprocessing/text.py:157: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [11]:
for i, sentences in enumerate(sentence_list):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            # convert sentence to words
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                # If the number of words in a sentence is less than 100 and it's index is < 20000 then assign it.
                if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                    data[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

#labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
#print('Shape of label tensor:', labels.shape)


Total 401391 unique tokens.
Shape of data tensor: (88002, 15, 60)


In [ ]:
# Get the input for a single document.

In [12]:
data_test_df = pd.read_csv('document similarity/document_similarity.csv',sep=',')
data_test_content = data_test_df.iloc[0].content

In [13]:
text = BeautifulSoup(data_test_content)
text = clean_str(text.get_text())
test_sentences = tokenize.sent_tokenize(text)
data_test = np.zeros((1, MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [15]:
i=0;
for j, sent in enumerate(test_sentences):
        if j < MAX_SENTS:
            # convert sentence to words
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                # If the number of words in a sentence is less than 100 and it's index is < 20000 then assign it.
                try:
                    if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                        data_test[i, j, k] = tokenizer.word_index[word]
                        k = k + 1
                except:
                    continue